In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
path='C:/Projects/Hackerearth/Problem 2/'

C:\Users\sharm_000\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pd.read_json(path+'train_data.json')
test_data=pd.read_json(path+'test_data.json')

In [3]:
train_data=data.transpose()
test=test_data.transpose()
train_data.shape,test.shape

((200000, 6), (100000, 5))

In [4]:
test=test.reset_index()
train_data=train_data.reset_index()

In [5]:
mappa={'neg':0,'pos':1}
train_data['segment']=train_data['segment'].map(mappa)

In [6]:
all_data=pd.concat([train_data,test],axis=0)
all_data=all_data.reset_index(drop=True)

In [7]:
all_data.columns=[u'cities', u'dow', u'genres', u'user_id', u'segment', u'titles', u'tod']

In [8]:
sum([int(i.split(':')[1] )for i in (all_data['cities'][0]).split(',')])

87386

In [9]:
[[i.split(':')[0],int(i.split(':')[1])] for i in all_data['dow'][0].split(',')]

[[u'1', 3412],
 [u'3', 15878],
 [u'2', 1737],
 [u'5', 10975],
 [u'4', 20974],
 [u'7', 17820],
 [u'6', 16580]]

In [10]:
all_data['no_of_Cities']=all_data['cities'].apply(lambda x:len(x.split(',')) )
all_data['no_of_dow']=all_data['dow'].apply(lambda x:len(x.split(',')) )
all_data['no_of_tod']=all_data['tod'].apply(lambda x:len(x.split(',')) )
all_data['no_of_genre']=all_data['genres'].apply(lambda x:len(x.split(',')) )
all_data['no_of_title']=all_data['titles'].apply(lambda x:len(x.split(',')) )
all_data['total_time_spent']=all_data['cities'].apply(lambda x:sum([int(i.split(':')[1] )for i in x.split(',')]))
all_data['dow']=all_data['dow'].apply(lambda x:[[i.split(':')[0],int(i.split(':')[1])] for i in x.split(',')])
all_data['tod']=all_data['tod'].apply(lambda x:[[i.split(':')[0],int(i.split(':')[1])] for i in x.split(',')])
all_data['genres']=all_data['genres'].apply(lambda x:[[i.split(':')[0],int(i.split(':')[1])] for i in x.split(',')])
all_data['no_of_titles']=all_data['titles'].apply(lambda x:len(x.split(',')))

In [11]:
def value_dow(x,a):
    try:
        return [i[1] for i in x if i[0]==a][0]
    except:
        return 0

In [ ]:
all_data['dow_1']=all_data['dow'].apply(lambda x:value_dow(x,'1'))
all_data['dow_2']=all_data['dow'].apply(lambda x:value_dow(x,'2'))
all_data['dow_3']=all_data['dow'].apply(lambda x:value_dow(x,'3'))
all_data['dow_4']=all_data['dow'].apply(lambda x:value_dow(x,'4'))
all_data['dow_5']=all_data['dow'].apply(lambda x:value_dow(x,'5'))
all_data['dow_6']=all_data['dow'].apply(lambda x:value_dow(x,'6'))
all_data['dow_7']=all_data['dow'].apply(lambda x:value_dow(x,'7'))

In [ ]:
for kk in range(0,24):
#     print kk
    all_data['tod_'+str(kk)]=all_data['tod'].apply(lambda x:value_dow(x,str(kk)))

In [ ]:
genre_list=set([item[0] for kk in all_data['genres'] for item in kk])

In [ ]:
for ee in genre_list:
#     print ee
    all_data['genres_'+str(ee)]=all_data['genres'].apply(lambda x:value_dow(x,str(ee)))

In [ ]:
naya_data=all_data.copy()
del naya_data['cities']
del naya_data['dow']
del naya_data['tod']
del naya_data['genres']
del naya_data['titles']
del naya_data['user_id']
del naya_data['segment']

In [ ]:
naya_data.shape

In [ ]:
target=train_data['segment']

In [34]:
model=xgb.XGBClassifier()

In [35]:
model.fit(naya_data[:200000], target)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [36]:
pred=model.predict_proba(naya_data[200000:])

In [39]:
pred[::,1]

array([ 0.02948648,  0.07009368,  0.19984329, ...,  0.00814223,
        0.01096414,  0.07363285], dtype=float32)

In [44]:
sub=pd.concat([test['index'],pd.DataFrame(pred[::,1])],axis=1)
sub.columns=['ID','segment']
sub.to_csv('prob2xgb.csv',index=False)

### RF

In [51]:
from sklearn import ensemble

In [55]:
rf1=ensemble.RandomForestClassifier(n_estimators=100)
rf1.fit(naya_data[:200000], target)
rfpred=rf1.predict_proba(naya_data[200000:])

In [56]:
sub2=pd.concat([test['index'],pd.DataFrame(rfpred[::,1])],axis=1)
sub2.columns=['ID','segment']
sub2.to_csv('prob2rf.csv',index=False)

### mean

In [59]:
np.corrcoef(sub['segment'],sub2['segment'])

array([[ 1.        ,  0.84781087],
       [ 0.84781087,  1.        ]])

In [63]:
sub3=(sub['segment']+sub2['segment'])/2

In [64]:
sub4=pd.concat([test['index'],pd.DataFrame(sub3)],axis=1)
sub4.columns=['ID','segment']
sub4.to_csv('prob2mean.csv',index=False)

### weighted

In [66]:
sub5=(.75*(sub['segment'])+.25*(sub2['segment']))

In [67]:
sub6=pd.concat([test['index'],pd.DataFrame(sub5)],axis=1)
sub6.columns=['ID','segment']
sub6.to_csv('prob2weighted.csv',index=False)